# Stage 4 Data Analysis on the Integrated Table

First, we repeat what we did in stage 3 and get a set of matching tuples.

In [138]:
import pandas as pd
import py_entitymatching as em
A = em.read_csv_metadata('./TableA_lower.csv', key = 'ID')
B = em.read_csv_metadata('./TableB_lower.csv', key = 'ID')

block_f = em.get_features_for_blocking(A, B)
block_t = em.get_tokenizers_for_blocking()
block_s = em.get_sim_funs_for_blocking()
r = em.get_feature_fn('jaccard(dlm_dc0(ltuple["Title"]), dlm_dc0(rtuple["Title"]))', block_t, block_s)
em.add_feature(block_f, 'Title_Title_jac_dlm_dc0_dlm_dc0', r)

ob = em.OverlapBlocker()
C = ob.block_tables(A, B, 'Author', 'Author', 
                    l_output_attrs=['Title','Author','Publication','Format','ISBN','Series', 'Physical Details'], 
                    r_output_attrs=['Title','Author','Publication','Format','ISBN','Series', 'Physical Details'], 
                    overlap_size = 2)

D = ob.block_candset(C, 'Title', 'Title', overlap_size = 4)

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


The table shows the corresponding attributes along with their respective types.
Please confirm that the information  has been correctly inferred.
If you would like to skip this validation process in the future,
please set the flag validate_inferred_attr_types equal to false.


,Left Attribute,Right Attribute,Left Attribute Type,Right Attribute Type,Example Features
0,ID,ID,short string (1 word),short string (1 word),Levenshtein Distance; Levenshtein Similarity
1,Title,Title,short string (1 word),short string (1 word),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
2,Author,Author,medium string (5 words to 10 words),short string (1 word to 5 words),Not Applicable: Types do not match
3,Publication,Publication,medium string (5 words to 10 words),short string (1 word to 5 words),Not Applicable: Types do not match
4,Format,Format,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
5,ISBN,ISBN,numeric,numeric,Exact Match; Absolute Norm
6,Series,Series,medium string (5 words to 10 words),medium string (5 words to 10 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
7,Physical Details,Physical Details,short string (1 word),short string (1 word to 5 words),Not Applicable: Types do not match


Do you want to proceed? (y/n):y


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [2]:
label_S = pd.read_csv('./Set_G.csv')
# em.copy_properties(S, label_S)
em.set_property(label_S, 'key', '_id')
em.set_property(label_S, 'fk_ltable', 'ltable_ID')
em.set_property(label_S, 'fk_rtable', 'rtable_ID')
label_S_rtable = em.read_csv_metadata('./label_S_rtable.csv')
label_S_ltable = em.read_csv_metadata('./label_S_ltable.csv')
em.set_property(label_S, 'rtable', label_S_rtable)
em.set_property(label_S, 'ltable', label_S_ltable)
label_S['ltable_Title'] = label_S['ltable_Title'].apply(lambda x: x.lower())
label_S['rtable_Title'] = label_S['rtable_Title'].apply(lambda x: x.lower())
label_S['ltable_Author'] = label_S['ltable_Author'].apply(lambda x: x.lower())
label_S['rtable_Author'] = label_S['rtable_Author'].apply(lambda x: x.lower())
label_S['ltable_Publication'] = label_S['ltable_Publication'].apply(lambda x: x.lower())
label_S['rtable_Publication'] = label_S['rtable_Publication'].apply(lambda x: x.lower())
label_S['ltable_Format'] = label_S['ltable_Format'].apply(lambda x: x.lower())
label_S['rtable_Format'] = label_S['rtable_Format'].apply(lambda x: x.lower())
label_S['ltable_Series'] = label_S['ltable_Series'].apply(lambda x: x.lower())
label_S['rtable_Series'] = label_S['rtable_Series'].apply(lambda x: x.lower())

In [8]:
match_f = em.get_features_for_matching(A, B)
match_t = em.get_tokenizers_for_matching()
match_s = em.get_sim_funs_for_matching()
f1 = em.get_feature_fn('jaccard(dlm_dc0(ltuple["Title"]), dlm_dc0(rtuple["Title"]))', match_t, match_s)
# f2 = em.get_feature_fn('jaccard(dlm_dc0(ltuple["Author"]), dlm_dc0(rtuple["Author"]))', match_t, match_s)
f3 = em.get_feature_fn('jaccard(dlm_dc0(ltuple["Publication"]), dlm_dc0(rtuple["Publication"]))', match_t, match_s)
f4 = em.get_feature_fn('jaccard(dlm_dc0(ltuple["Series"]), dlm_dc0(rtuple["Series"]))', match_t, match_s)
em.add_feature(match_f, 'Title_Title_jac_dlm_dc0_dlm_dc0', f1)
# em.add_feature(match_f, 'Author_Author_jac_dlm_dc0_dlm_dc0', f2)
em.add_feature(match_f, 'Publication_Publication_jac_dlm_dc0_dlm_dc0', f3)
em.add_feature(match_f, 'Series_Series_jac_dlm_dc0_dlm_dc0', f4)


# Add blackbox feature

import re
# for Roman numerals matching
def Title_Title_blackbox_1(x, y):
    
    # get name attribute
    x_title = x['Title']
    y_title = y['Title']
#     regex_roman = '\s+[MDCLXVI]+\s+'
    regex_roman = '\s+[mdclxvi]+($|\s+)'
    x_match = None
    y_match = None
    if re.search(regex_roman, x_title):
        x_match = re.search(regex_roman, x_title).group(0).strip()
    if re.search(regex_roman, y_title):
        y_match = re.search(regex_roman, y_title).group(0).strip()

    if x_match is None or y_match is None:
        return False
    else:
        return x_match == y_match

em.add_blackbox_feature(match_f, 'blackbox_1', Title_Title_blackbox_1)


# for number matching (e.g. 6th edition)
def Title_Title_blackbox_2(x, y):
    # x, y will be of type pandas series
    
    x_title = x['Title']
    y_title = y['Title']
    regex_number = '\s+(\d+)\s*th'
    x_match = None
    y_match = None
    if re.search(regex_number, x_title):
        x_match = re.search(regex_number, x_title).group(1)
    if re.search(regex_number, y_title):
        y_match = re.search(regex_number, y_title).group(1)

    if x_match is None or y_match is None:
        return False
    else:
        return x_match == y_match

em.add_blackbox_feature(match_f, 'blackbox_2', Title_Title_blackbox_2)

# for number matching (e.g. 6th edition)
from fuzzywuzzy import fuzz
def Author_Author_blackbox_3(x, y):
    # x, y will be of type pandas series
    
    x_author = x['Author']
    y_author = y['Author']
    return fuzz.token_set_ratio(x_author, y_author)/100.0
    
em.add_blackbox_feature(match_f, 'blackbox_3', Author_Author_blackbox_3)

The table shows the corresponding attributes along with their respective types.
Please confirm that the information  has been correctly inferred.
If you would like to skip this validation process in the future,
please set the flag validate_inferred_attr_types equal to false.


,Left Attribute,Right Attribute,Left Attribute Type,Right Attribute Type,Example Features
0,ID,ID,short string (1 word),short string (1 word),Levenshtein Distance; Levenshtein Similarity
1,Title,Title,short string (1 word),short string (1 word),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
2,Author,Author,medium string (5 words to 10 words),short string (1 word to 5 words),Not Applicable: Types do not match
3,Publication,Publication,medium string (5 words to 10 words),short string (1 word to 5 words),Not Applicable: Types do not match
4,Format,Format,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
5,ISBN,ISBN,numeric,numeric,Exact Match; Absolute Norm
6,Series,Series,medium string (5 words to 10 words),medium string (5 words to 10 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
7,Physical Details,Physical Details,short string (1 word),short string (1 word to 5 words),Not Applicable: Types do not match


Do you want to proceed? (y/n):y


True

In [9]:
match_f = match_f[(match_f['left_attribute'] != 'ID') & (match_f['left_attribute'] != 'ISBN')]
match_f = match_f[(match_f['left_attribute'] != 'Format') & (match_f['left_attribute'] != 'Series')]

In [11]:
H = em.extract_feature_vecs(label_S, feature_table=match_f, attrs_after=['label'])

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [12]:
# RF
rf = em.RFMatcher(n_estimators = 300,max_depth = 300, name='RF')
rf.fit(table=H, 
       exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
       target_attr='label')



In [13]:
H_test = em.extract_feature_vecs(D, feature_table=match_f)
pred_table = rf.predict(table= H_test, 
                        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID'], 
                        target_attr='predicted_labels', 
                        return_probs=True, 
                        probs_attr='proba', 
                        append=True)
# eval_summary = em.eval_matches(pred_table, 'label', 'predicted_labels')
# eval_summary

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


In [14]:
pred_rows = pred_table[pred_table['predicted_labels'] == 1]

## Merge the tables

Here we obtained the matching set based on the prediction.


There are 1337 matched books in total.

In [20]:
matched_set = D[D['_id'].isin(pred_rows['_id'])]

In [133]:
matched_set

,_id,ltable_ID,rtable_ID,ltable_Title,ltable_Author,ltable_Publication,ltable_Format,ltable_ISBN,ltable_Series,ltable_Physical Details,rtable_Title,rtable_Author,rtable_Publication,rtable_Format,rtable_ISBN,rtable_Series,rtable_Physical Details
0,0,a2337,b0,"""statistical learning and data science""","""edited by mireille gettler summa ... [and others]""","""boca raton : crc press, [2012] ©2012""","""books""",9781439867631,"""series in computer science and data analysis,""","""nan""","""statistical learning and data science""","""summa, mireille gettler.""","""boca raton""","""nan""",9781439867631,"""series in computer science and data analysis.""","""xv, 227 p."""
1,1,a3097,b1,"""intelligent techniques for data science""","""rajendra akerkar, priti srinivas sajja""","""cham, switzerland : springer, 2016.""","""books""",9783319292069,"""nan""","""nan""","""intelligent techniques for data science""","""akerkar, rajendra""","""cham, switzerland""","""electronic books.""",9783319292069,"""nan""","""1 online resource (xvi, 272 pages)"""
2,2,a4508,b2,"""algorithms for data science""","""brian steele, john chandler, swarna reddy""","""cham, switzerland : springer, 2016.""","""books""",9783319457970,"""nan""","""nan""","""algorithms for data science""","""steele, brian""","""cham, switzerland""","""electronic books.""",9783319457970,"""nan""","""1 online resource (xxiii, 430 pages)"""
3,3,a3267,b3,"""data science at the command line""","""jeroen janssens""","""first edition. sebastopol, ca : o'reilly, 2014. ©2015""","""books""",9781491947852,"""nan""","""nan""","""data science at the command line""","""janssens, jeroen""","""sebastopol, ca""","""nan""",9781491947852,"""nan""","""xvii, 191 pages"""
5,5,a4755,b5,"""introduction to hpc with mpi for data science""","""frank nielsen""","""cham : springer, 2016.""","""books""",9783319219035,"""undergraduate topics in computer science,""","""nan""","""introduction to hpc with mpi for data science""","""nielsen, frank""","""cham""","""electronic books.""",9783319219035,"""undergraduate topics in computer science, 1863-7310""","""1 online resource (xxxiii, 282 pages)"""
6,6,a2523,b6,"""data science using oracle data miner and oracle r enterprise : transform your business systems ...","""sibanjan das""","""berkeley, ca : apress, 2016. berkeley, ca : apress, 2016.""","""books""",9781484226148,"""nan""","""nan""","""data science using oracle data miner and oracle r enterprise : transform your business systems ...","""das, sibanjan.""","""berkeley, ca""","""electronic books.""",9781484226148,"""nan""","""1 online resource (300 pages)"""
7,7,a3584,b7,"""the data science handbook""","""field cady""","""hoboken, nj : john wiley & sons, inc., 2017.""","""books""",9781119092933,"""nan""","""nan""","""the data science handbook""","""cady, field, 1984-""","""hoboken, nj""","""electronic books. | handbooks and manuals.""",9781119092933,"""nan""","""1 online resource"""
8,8,a1869,b8,"""data science : create teams that ask the right questions and deliver real value""","""doug rose""","""[berkeley, ca] : apress, 2016.""","""books""",9781484222539,"""nan""","""nan""","""data science : create teams that ask the right questions and deliver real value""","""rose, doug, (agile coach)""","""[berkeley, ca]""","""electronic books.""",9781484222539,"""nan""","""1 online resource"""
9,9,a1186,b9,"""spatial big data science classification techniques for earth observation imagery""","""zhe jiang, shashi shekhar""","""cham : springer, 2017.""","""books""",9783319601953,"""nan""","""nan""","""spatial big data science : classification techniques for earth observation imagery""","""jiang, zhe.""","""cham""","""electronic books.""",9783319601953,"""nan""","""1 online resource"""
10,10,a3432,b11,"""practical data science cookbook : 89 hands-on recipes to help you complete real-world data scie...","""tony ojeda, sean patrick murphy, benjamin bengfort, abhijit dasgupta""","""birmingham, uk : packt publishing ltd., september 20

In [129]:
matched_set.shape

(1337, 17)

In [26]:
matched_id = matched_set[['ltable_ID', 'rtable_ID']]

We put the id of matching tuples in to dictionaries.

In [37]:
# AB_list=[['ltable_ID','rtable_ID'] for x in pred_table if predicted_labels==1]
AB_list = matched_id.values.tolist()
AB_dict =  {item[0]: item[1] for item in AB_list}
BA_dict =  {item[1]: item[0] for item in AB_list}

Here we merge the matching tuples.

For each field of a matching tuple, we simply keep the one with longer string length.

In [139]:
df = matched_set.iloc[:, 3:10]

In [140]:
df = df.rename(columns={'ltable_Title' : 'Title',
                  'ltable_Author' : 'Author',
                   'ltable_Publication' : 'Publication',
                   'ltable_Format' : 'Format',
                   'ltable_ISBN' : 'ISBN',
                   'ltable_Series' : 'Series',
                   'ltable_Physical Details' : 'Physical Details'
                  })

In [141]:
attr=['Title','Author','Publication','Format','ISBN','Series', 'Physical Details']
i = 0
for index, r in matched_set.iterrows():
    for  a in attr:
        left_name = 'ltable_' + a
        right_name = 'rtable_' + a
#         print(index,' ' ,attr.index(a))
        if len(str(r[left_name])) < len(str(r[right_name])):
            
            df.iloc[i, attr.index(a)] = r[right_name]
        else:
            df.iloc[i, attr.index(a)] = r[left_name]
    i += 1

We added an Source attribute to indicate the source of a tuple.

In [145]:
df['Source'] = 'ab'
A['Source'] = 'a'
B['Source'] = 'b'

After merging the matching set, we concat the set with the original table A and B.

In [146]:
set_merged = pd.concat([df, A[~A['ID'].isin(AB_dict.keys())].iloc[:,1:], B[~B['ID'].isin(BA_dict.keys())].iloc[:,1:]])

In [147]:
set_merged

,Title,Author,Publication,Format,ISBN,Series,Physical Details,Source
0,"""statistical learning and data science""","""edited by mireille gettler summa ... [and others]""","""boca raton : crc press, [2012] ©2012""","""books""",9781439867631,"""series in computer science and data analysis,""","""xv, 227 p.""",ab
1,"""intelligent techniques for data science""","""rajendra akerkar, priti srinivas sajja""","""cham, switzerland : springer, 2016.""","""electronic books.""",9783319292069,"""nan""","""1 online resource (xvi, 272 pages)""",ab
2,"""algorithms for data science""","""brian steele, john chandler, swarna reddy""","""cham, switzerland : springer, 2016.""","""electronic books.""",9783319457970,"""nan""","""1 online resource (xxiii, 430 pages)""",ab
3,"""data science at the command line""","""janssens, jeroen""","""first edition. sebastopol, ca : o'reilly, 2014. ©2015""","""books""",9781491947852,"""nan""","""xvii, 191 pages""",ab
5,"""introduction to hpc with mpi for data science""","""nielsen, frank""","""cham : springer, 2016.""","""electronic books.""",9783319219035,"""undergraduate topics in computer science, 1863-7310""","""1 online resource (xxxiii, 282 pages)""",ab
6,"""data science using oracle data miner and oracle r enterprise : transform your business systems ...","""das, sibanjan.""","""berkeley, ca : apress, 2016. berkeley, ca : apress, 2016.""","""electronic books.""",9781484226148,"""nan""","""1 online resource (300 pages)""",ab
7,"""the data science handbook""","""cady, field, 1984-""","""hoboken, nj : john wiley & sons, inc., 2017.""","""electronic books. | handbooks and manuals.""",9781119092933,"""nan""","""1 online resource""",ab
8,"""data science : create teams that ask the right questions and deliver real value""","""rose, doug, (agile coach)""","""[berkeley, ca] : apress, 2016.""","""electronic books.""",9781484222539,"""nan""","""1 online resource""",ab
9,"""spatial big data science : classification techniques for earth observation imagery""","""zhe jiang, shashi shekhar""","""cham : springer, 2017.""","""electronic books.""",9783319601953,"""nan""","""1 online resource""",ab
10,"""practical data science cookbook : 89 hands-on recipes to help you complete real-world data scie...","""tony ojeda, sean patrick murphy, benjamin bengfort, abhijit dasgupta""","""birmingham, uk : packt publishing ltd., september 2014. ©2014""","""electronic books.""",9781783980246,"""nan""","""1 online resource""",ab


Final merged set contains 9350 tuples in total.

In [148]:
set_merged.shape

(9350, 8)

We save the set to Table_E.csv.

In [149]:
set_merged.to_csv('Table_E.csv', sep=',')